In [1]:
import numpy as np
import matplotlib.pyplot as plt
import time

In [2]:
"""SCHEDULE PART"""

def refScenarios(N_ref,typ,arguments=None): 
        
    if arguments is not None:
        mu,m,L = arguments[0],arguments[1],arguments[2]
    else:
        mu,m,L = 0,0,1
        
    if typ=='FGD':
        s = mu+m
        A_list = []
        Ak = 0
        for _ in range(N_ref):
            ak_plus = ((1+Ak*s)+((1+Ak*s+4*Ak*L)**(1/2) * (1+Ak*s)**(1/2)))/(2*L)
            Ak += ak_plus
            A_list.append(Ak)
        A_array = np.array(A_list) 
        a_ref,b_ref = A_array/A_array[-1],np.ones(N_ref)
        
    elif typ=='iAFB': # naive for b_ref here :) 
        
        if arguments is not None:
            mu,L,zeta,sigma = arguments[0],arguments[1],arguments[2],arguments[3]
        else:
            mu,L,zeta,sigma = 0,1,0,0
           
        A_list = []
        Ak = 0
        for _ in range(N_ref):
            stepsize = (1-sigma**2)/L
            eta = (1-zeta**2)*stepsize
            ak = (eta + 2*Ak*mu*eta + np.sqrt(4*eta*Ak*(1+mu*eta)*(1+Ak*mu)+ eta**2))/2
            Ak += ak
            A_list.append(Ak)
        A_array = np.array(A_list) 
        a_ref,b_ref = A_array/A_array[-1],np.ones(N_ref)
        
    elif typ=='GD':
            
        conditions = (1-np.ones(N_ref)*(mu+m)/(L+m))
        buf = np.cumprod(conditions)[::-1]
        a_ref,b_ref = buf/np.sum(buf),np.ones(N_ref)
    
    else: # neutral 
        
        a_ref,b_ref = np.ones(N_ref),np.ones(N_ref)
        
    return a_ref,b_ref

def reaffect(pre_vec,index):
    vec = np.zeros(pre_vec.shape)
    for idx,value in enumerate(pre_vec):
        vec[index[idx]] = value
    return vec

def refAccuracyScheduler(N_ref,M,bar_delta,r,arguments=None,typ='FGD',verbose=False):
    
    a_ref,b_ref = refScenarios(N_ref,typ,arguments)
    
    nu_sorted = np.sort(b_ref/a_ref)[::-1] # descending order
    
    indices_ = np.argsort(b_ref/a_ref)[::-1]
    
    a,b = a_ref[indices_],b_ref[indices_]
    
    buf = np.cumsum((b*a**r)**(1/(r+1)))
    max_buf = buf[-1]
    buf_b = np.cumsum(b)
    max_buf_b = buf_b[-1]
    
    def lh(NB,r):
        
        if r>0:
            if NB==0:
                return bar_delta*(max_buf/max_buf_b)**(1/r)
            return bar_delta*((max_buf-buf[NB-1])/(max_buf_b-buf_b[NB-1]*M**(-r)))**(1/r)
        elif r==0:
            if NB==0:
                return bar_delta*np.prod(nu_sorted**(-b/max_buf_b))
            return bar_delta*np.prod(nu_sorted[NB:]**(-b[NB:]/(max_buf_b-buf_b[NB-1])))/M**(buf_b[NB-1]/(max_buf_b-buf_b[NB-1]))
        else: 
            return 0
        
    hat_lambda = lh(0,r)
    
    if nu_sorted[0]**(1/(r+1)) * hat_lambda < bar_delta*M:
        if verbose:
            print('immediate return')
        return reaffect(hat_lambda * nu_sorted**(1/(r+1)),indices_)
        
    
    Nb_low,Nb_high = 0,N_ref-1
    
    while Nb_high-Nb_low>1:
        
        Nb_try = int(np.ceil((Nb_high+Nb_low)/2))
        hat_lambda = lh(Nb_try,r)
        
        if hat_lambda*nu_sorted[Nb_try-1]**(1/(r+1))>=M*bar_delta:
            if hat_lambda*nu_sorted[Nb_try]**(1/(r+1))<=M*bar_delta:
                if verbose:
                    print('ok outcome')
                return reaffect(np.concatenate((np.ones(Nb_try)*M*bar_delta,hat_lambda*nu_sorted[Nb_try:]**(1/(r+1)))),indices_)
            else:
                Nb_low = Nb_try
        else: 
            Nb_high = Nb_try
    
    hat_lambda_high = lh(Nb_high,r)
    if hat_lambda_high*nu_sorted[Nb_high-1]**(1/(r+1))>=M*bar_delta and hat_lambda_high*nu_sorted[Nb_high]**(1/(r+1)) <= M*bar_delta:
        if verbose:
            print('ok high')
        return reaffect(np.concatenate((np.ones(Nb_high)*M*bar_delta,hat_lambda_high*nu_sorted[Nb_high:]**(1/(r+1)))),indices_)
    hat_lambda_low = lh(Nb_low,r)
    if verbose:
        print('ok low (guessed)')
    return reaffect(np.concatenate((np.ones(Nb_low)*M*bar_delta,hat_lambda_low*nu_sorted[Nb_low:]**(1/(r+1)))),indices_)

$$\Bigg(\frac{\delta_{k_{2}}}{\delta_{k_{1}}}\Bigg)^{\hat{r}_{1,2}} = \frac{\omega_{k_1} / b_{k_{1}}}{\omega_{k_2} / b_{k_{2}}} \Rightarrow \hat{r}_{1,2} = \log_{\frac{\delta_{k_{2}}}{\delta_{k_{1}}}}\Big(\frac{\omega_{k_1} / b_{k_{1}}}{\omega_{k_2} / b_{k_{2}}}\Big) = \frac{\log\bigg(\frac{\omega_{k_2} / b_{k_{2}}}{\omega_{k_1} / b_{k_{1}}}\bigg)}{\log\bigg(\frac{\delta_{k_{1}}}{\delta_{k_{2}}}\bigg)}$$